
# Advanced Scheduling of Slices

This notebook demonstrates how to schedule slices in advance by specifying a future time range for resource reservation. 
Advanced scheduling allows you to request resources for a specific interval in the future and ensures that the requested resources are 
available for the required duration within that window.

The notebook covers how to:
- Specify a time range for the slice reservation.
- Set a lease duration (in hours) to indicate how long the resources should be available.
- Submit the slice.


## Import the FABlib Library

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();


## Set the parameters

In [ ]:
# Define parameters (modify these as needed)
slice_name = "AdvancedSchedulingSlice-1"  # Name for the new slice

node1_name = 'Node1'
node2_name = 'Node2'

network1_name='net1'

model = "NIC_ConnectX_6"

site1 = "RENC"
site2 = "UKY"

### Define the Time Duration
Define the duration over which resource availability will be evaluated. This duration represents the continuous period of time for which resources must be available to meet the reservation requirements. For example, in the following code snippet, a 6-hour reservation window is specified, meaning the system will search within the given `start` and `end` time range to identify a contiguous 6-hour block where resources are available for the requested slice.

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

start = (datetime.now(timezone.utc) + timedelta(hours=2))
end = start + timedelta(days=3)
lease_in_hours = 6

## Create the Experiment Slice

In this example, we will create a slice with nodes configured with `NIC_ConnectX_6` NICs and establish network connections between them.

Dedicated NICs may not always be available at the desired sites, so we are requesting a slice within a specified future time interval for a set duration.

Once a future slice is allocated, it will enter the `AllocatedOK` state. The resources in the slice will then be provisioned and become active at the designated lease start time.

In [ ]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

# Network
net1 = slice.add_l2network(name=network1_name, subnet="192.168.1.0/24")

node1 = slice.add_node(name=node1_name, site=site1)
node1_iface1 = node1.add_component(model=model, name='nic1').get_interfaces()[0]

node1_iface1.set_mode('auto')
net1.add_interface(node1_iface1)

node2 = slice.add_node(name=node2_name, site=site2)
node2_iface1 = node2.add_component(model=model, name='nic1').get_interfaces()[0]

node2_iface1.set_mode('auto')
net1.add_interface(node2_iface1)

slice.submit(lease_start_time=start, lease_end_time=end, lease_in_hours=lease_in_hours)

## Run the Experiment

The experiment can begin once the lease start time arrives, and the resources are fully provisioned and reach an active state.

At that point, verify connectivity across the Layer 2 Wide Area Network.

In [ ]:
slice = fablib.get_slice(slice_name)

node1 = slice.get_node(name=node1_name)        
node2 = slice.get_node(name=node2_name)           

node2_addr = node2.get_interface(network_name=network1_name).get_ip_addr()

stdout, stderr = node1.execute(f'ping -c 5 {node2_addr}')

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice.delete()